# Programming Assignment 2

In this assignment, you will:

1. Practice writing code to train and evaluate models using both the two-way holdout method, and an evaluation approach appropriate for models with hyperparameters that uses k-fold cross validation plus a test set.

2. Practice writing code to optimize a machine learning model. In particular, we will use gradient descent to optimize a logistic regression model.

3. **574 Only**: Perform optimization with a different algorithm (Newton-Raphson)

# Resources you can use to complete this assignment (a COMPLETE list)

**NOTE: You ARE allowed to use Google to find things that fit this list (i.e. it is often easy to google something like "plotly draw line graph" to find the right part of the plotly documentation).**

- Anything linked to in this article
- Anything linked to from the course web page
- Any materials from another online course taught at a university (**if you use this, you MUST provide a link to the exact document used**)
- Anything posted by Kenny, Navid, or Yincheng on Piazza



# Grading

For grading of code in Part 2, we will execute the submitted notebook as follows:

```shell
jupyter nbconvert --to python Assignment2-Student.ipynb
python Assignment2-Student.py
```

The PDF that comes along with this document has other details on the points awarded for each part.

As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 4 things:
- Your completed jupyter notebook.
- Your written report, answering all questions asked here (and copied in the assignment PDF)
- `part_1.1_results.csv`
- `part_1.4_results.csv`


# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? 
- **1.1.2** Which model had the best RMSE on the *test data*? 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)*
- **1.1.4** What can we say about the utility of the Spotify features based on these results?

In [1]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

result_data = []

all_features = CONTINUOUS_FEATURES + CATEGORICAL_FEATURES

train_data = pd.read_csv("part1_train.csv")
Ytrain = train_data["score"]
print("Ytrain shape:", Ytrain.shape)
test_data = pd.read_csv("part1_test.csv") 
Ytest = test_data["score"]
print("Ytest shape:", Ytest.shape)

models_data = {"feature_set":[], "train_rmse":[], "test_rmse":[]}
print(models_data)

for feature_set in feature_sets:
    # Write your code for Part 1.1 here!
    catfs = []
    contfs = []

    for feature in feature_set:
      if (feature in CATEGORICAL_FEATURES): catfs.append(feature)
      elif (feature in CONTINUOUS_FEATURES): contfs.append(feature)

    linreg_pipeline = make_pipeline(ColumnTransformer([('categorical', OneHotEncoder(), catfs), 
                                                       ('numerical', StandardScaler(), contfs)]), 
                                                      LinearRegression())
    Xtrain = train_data[feature_set]
    Xtest = test_data[feature_set]

    linreg_pipeline.fit(Xtrain, Ytrain)
    train_pred = np.asarray(linreg_pipeline.predict(Xtrain))
    test_pred = np.asarray(linreg_pipeline.predict(Xtest))

    train_error = np.square(train_pred - Ytrain)
    test_error = np.square(test_pred - Ytest)
    train_rmse = np.sqrt(np.sum(train_error) / len(Xtrain))
    test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))

    models_data["feature_set"].append("_".join(feature_set))
    models_data["train_rmse"].append(train_rmse)
    models_data["test_rmse"].append(test_rmse)

    pass
results_data = pd.DataFrame(models_data)
print(results_data)
results_data.to_csv("madhavi_part1.1_res.csv")
results_data.feature_set[-1:]


Ytrain shape: (16270,)
Ytest shape: (1808,)
{'feature_set': [], 'train_rmse': [], 'test_rmse': []}
                                          feature_set  train_rmse  test_rmse
0                                              artist    1.226947   1.243599
1                                        reviewauthor    1.181260   1.195335
2                                         releaseyear    1.235994   1.232989
3                                         recordlabel    1.204325   1.234508
4                                               genre    1.234395   1.234444
5                                        danceability    1.240362   1.241097
6                                              energy    1.240900   1.239929
7                                                 key    1.243497   1.243823
8                                            loudness    1.236968   1.237578
9                                         speechiness    1.243530   1.243566
10                                       acousticness 

17    artist_reviewauthor_releaseyear_recordlabel_ge...
Name: feature_set, dtype: object

In addition, please answer the following questions:

1.1.1 Which model had the best RMSE on the training data?

In [4]:
results_data.feature_set[np.argmin(results_data.train_rmse)]

'artist_reviewauthor_releaseyear_recordlabel_genre_danceability_energy_key_loudness_speechiness_acousticness_instrumentalness_liveness_valence_tempo'


1.1.2 Which model had the best RMSE on the test data?


In [5]:
results_data.feature_set[np.argmin(results_data.test_rmse)]

'artist_reviewauthor_releaseyear_recordlabel_genre_danceability_energy_key_loudness_speechiness_acousticness_instrumentalness_liveness_valence_tempo'

1.1.3 Which feature do you believe was the most important one? Why? (Note: There is more than one perfectly acceptable way to answer this question)


In [6]:
# Considering 0-14 feature sets, which has individual features
subdata = results_data[:15]
subdata.feature_set[np.argmin(subdata.test_rmse)]


'reviewauthor'

1.1.4 What can we say about the utility of the Spotify features based on these results?

Ans: Soptify features are not contributing much to the model.

## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [7]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso

# Do the CV to find alpha
features = feature_sets[-1]

# Retrain the model
train_data = pd.read_csv("part1_train.csv")
Ytrain = train_data["score"]
test_data = pd.read_csv("part1_test.csv") 
Ytest = test_data["score"]

catfs = []
contfs = []

for feature in features:
  if (feature in CATEGORICAL_FEATURES): catfs.append(feature)
  elif (feature in CONTINUOUS_FEATURES): contfs.append(feature)

col_pipeline = make_pipeline(ColumnTransformer([('categorical', OneHotEncoder(drop="if_binary", sparse=False), catfs), 
                                                    ('numerical', StandardScaler(), contfs)]))
linreg_pipeline = make_pipeline(col_pipeline, LassoCV(cv=10, random_state=1))

Xtrain = train_data[features]
crossval_result = linreg_pipeline.fit(Xtrain, Ytrain)

1.2.1 - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.

In [8]:
col_pipeline.named_steps['columntransformer'].named_transformers_['categorical'].get_feature_names_out().shape[0]


680

1.2.2 - What was the best alpha value according to your cross-validation results?

In [9]:

best_alpha = linreg_pipeline.steps[-1][1].alpha_
print("Best alpha:", best_alpha)

Best alpha: 0.0001685701255211163


1.2.3 - What was the average RMSE of the model with this alpha value on the k-fold cross validation on the training data?


In [10]:
mean_rmses = np.sqrt(crossval_result[1].mse_path_.mean(axis=1))
min_rmse_at = np.argmin(mean_rmses)
print("Mean RMSE for best alpha:", mean_rmses[min_rmse_at])

Mean RMSE for best alpha: 1.1603036173033567


1.2.4 - What was the RMSE of the model with this alpha value on the k-fold cross validation on the test data?

In [11]:
test_pred = np.asarray(linreg_pipeline.predict(Xtest))
test_error = np.square(test_pred - Ytest)
test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))
print("test_rmse:", test_rmse)


test_rmse: 1.1649797846303533


In [12]:
# ALTERNATE WAY
features = feature_sets[-1]
train_data = pd.read_csv("part1_train.csv")
Ytrain = train_data["score"]
test_data = pd.read_csv("part1_test.csv") 
Ytest = test_data["score"]
Xtrain = train_data[features]
Xtest = test_data[features]

best_linreg_pipeline = make_pipeline(col_pipeline, Lasso(alpha=best_alpha))
best_alpha_result = best_linreg_pipeline.fit(Xtrain, Ytrain)

test_pred = np.asarray(best_linreg_pipeline.predict(Xtest))
test_error = np.square(test_pred - Ytest)
test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))
print("test_rmse", test_rmse)



test_rmse 1.1649797846303533


Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.
- **1.2.2** - What was the best `alpha` value according to your cross-validation results?
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data?
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data?


# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model?
- **1.3.2** - What percentage of the coefficients are non-zero in this final model?
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know?
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know?
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?


1.3.1 - How many non-zero coefficients are in this final model?

In [13]:
print("#non-zero coeffs:", np.count_nonzero(best_linreg_pipeline[-1].coef_))

#non-zero coeffs: 484


1.3.2 - What percentage of the coefficients are non-zero in this final model?

In [14]:
print("% of non-zero coeffs:", float(np.count_nonzero(best_linreg_pipeline[-1].coef_))/float(len(best_linreg_pipeline[-1].coef_))*100, "%")

% of non-zero coeffs: 70.04341534008684 %


1.3.3 - Who were the three most critical review authors, as estimated by the model? How do you know?

In [15]:
lasso_results = pd.DataFrame({"feature_names" : best_alpha_result[-2].get_feature_names_out(),
              "coefs" : best_alpha_result[-1].coef_
             })
lasso_results['abs_coef'] = np.abs(lasso_results.coefs)

print("1.3.3:   3 most critical review authors:")
print(lasso_results[lasso_results.feature_names.str.contains("reviewauthor")].sort_values("coefs",ascending=True)[0:3].feature_names)
# Reason: Because these authors have the highest negative coefficients values.

# Below one is also accepted.
#print(lasso_results[lasso_results.feature_names.str.contains("reviewauthor")].sort_values("abs_coef",ascending=False)[0:3])
# Reason: Because these authors have the highest absolute coefficients values.

1.3.3:   3 most critical review authors:
112    categorical__reviewauthor_Alison Fields
177    categorical__reviewauthor_Evan McGarvey
137      categorical__reviewauthor_Brian James
Name: feature_names, dtype: object


1.3.4 - Who were the three artists that reviewers tended to like the most? How do you know?

In [16]:
print("1.3.4:   3 most trending artists:")
print(lasso_results[lasso_results.feature_names.str.contains("artist")].sort_values("coefs",ascending=False)[0:3])

#Reason: Because these artists had the highest coefficient values.

1.3.4:   3 most trending artists:
                      feature_names     coefs  abs_coef
70       categorical__artist_R.E.M.  0.981431  0.981431
52  categorical__artist_Miles Davis  0.906630  0.906630
42   categorical__artist_Kanye West  0.571318  0.571318


1.3.5 - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?

In [17]:
print("Most Liked reviewer:", lasso_results[lasso_results.feature_names.str.contains("genre")].sort_values("coefs",ascending=False)[0:1].feature_names)
#Reason: Because these reviewers had the highest coefficient values.

print("Most Disliked reviewer:", lasso_results[lasso_results.feature_names.str.contains("genre")].sort_values("coefs",ascending=True)[0:2].feature_names)
#Reason: Because these reviewers had the least coefficient values.

Most Liked reviewer: 672    categorical__genre_Global
Name: feature_names, dtype: object
Most Disliked reviewer: 679           categorical__genre_nan
669    categorical__genre_Electronic
Name: feature_names, dtype: object


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance
- **1.4.2** Which model performed the best overall on the cross-validation? 
- **1.4.3** Which model performed the best overall on the final test set? 
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*)

In [18]:
# 1.4 Codes:

# Read in data:

data_folds = dict()
for i in range(5):
  fname = "1.2_fold" + str(i) + ".csv"
  data_folds[i] = pd.read_csv(fname)

total_train_data = pd.read_csv("part1_train.csv")
total_Ytrain = total_train_data["score"]
total_test_data = pd.read_csv("part1_test.csv") 
total_Ytest = total_test_data["score"]

features = feature_sets[-1]
total_Xtrain = total_train_data[features]
total_Xtest = total_test_data[features]

# Make pipeline for features data processing
col_pipeline = make_pipeline(ColumnTransformer([('numerical', StandardScaler(), CONTINUOUS_FEATURES),
          ('categorical',  OneHotEncoder(handle_unknown = 'ignore'), CATEGORICAL_FEATURES)]))


In [19]:
# type(data_folds[0])

# for nfold in range(5):

#   train_set = pd.DataFrame()
#   val_set = pd.DataFrame()

#   for i in range(5):
#     if (i != nfold):
#       train_set = pd.concat([train_set, data_folds[i]], axis=0)
#     else:
#       val_set = pd.concat([val_set, data_folds[i]], axis=0)
#   print(train_set.shape)
#   print(test_set.shape)

In [20]:
# Store results in frame
results_data = {"model_name":[], "hyperparamter_setting":[],"mean_training_rmse":[], "sd_training_rmse":[], "test_rmse":[]}
all_features = CONTINUOUS_FEATURES + CATEGORICAL_FEATURES

## DECISION TREE REG

from sklearn.tree import DecisionTreeRegressor

depths = [5, 10, 20]

for depth in depths:

  model_name = "DTR"
  hyp = str(depth)
  train_rmses = []

  for nfold in range(5):
  
    train_set = pd.DataFrame()
    val_set = pd.DataFrame()

    for i in range(5):
      if (i != nfold):
        train_set = pd.concat([train_set, data_folds[i]], axis=0)
      else:
        val_set = pd.concat([val_set, data_folds[i]], axis=0)

    Xtrain = train_set[all_features]
    Ytrain = train_set["score"]
    Xval = val_set[all_features]
    Yval = val_set["score"]

    
    pipeline = make_pipeline(col_pipeline, DecisionTreeRegressor(criterion="squared_error", max_depth=depth))
    pipeline.fit(Xtrain, Ytrain)

    val_pred = np.asarray(pipeline.predict(Xval))
    val_error = np.square(val_pred - Yval)
    val_rmse = np.sqrt(np.sum(val_error) /len(Xval))
    train_rmses.append(val_rmse)
  
  mean_train_rmse = np.mean(train_rmses)
  sd_train_rmse = np.std(train_rmses)

  results_data["model_name"].append(model_name)
  results_data["hyperparamter_setting"].append(hyp)
  results_data["mean_training_rmse"].append(mean_train_rmse)
  results_data["sd_training_rmse"].append(sd_train_rmse)

  #print(depth)
  pipeline = make_pipeline(col_pipeline, DecisionTreeRegressor(criterion="squared_error", max_depth=depth))
  pipeline.fit(total_Xtrain, total_Ytrain)
  test_pred = np.asarray(pipeline.predict(total_Xtest))
  test_error = np.square(test_pred - total_Ytest)
  test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))
  results_data["test_rmse"].append(test_rmse)




In [21]:
print(results_data)
dfs = pd.DataFrame(results_data)
dfs

{'model_name': ['DTR', 'DTR', 'DTR'], 'hyperparamter_setting': ['5', '10', '20'], 'mean_training_rmse': [1.208094186519705, 1.2454427211400911, 1.3349907701785437], 'sd_training_rmse': [0.02330679904192824, 0.02597186157714203, 0.015919245228671112], 'test_rmse': [1.2021076225285747, 1.2622793067391083, 1.372671998450443]}


,model_name,hyperparamter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,DTR,5,1.208094,0.023307,1.202108
1,DTR,10,1.245443,0.025972,1.262279
2,DTR,20,1.334991,0.015919,1.372672


In [22]:
## RIDGE REGRESSION

from sklearn.linear_model import Ridge
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]

for ld in lambdas:
  model_name = "ridge"
  hyp = str(ld)

  train_rmses = []

  for nfold in range(5):
  
    train_set = pd.DataFrame()
    val_set = pd.DataFrame()

    for i in range(5):
      if (i != nfold):
        train_set = pd.concat([train_set, data_folds[i]], axis=0)
      else:
        val_set = pd.concat([val_set, data_folds[i]], axis=0)

    Xtrain = train_set[all_features]
    Ytrain = train_set["score"]
    Xval = val_set[all_features]
    Yval = val_set["score"]

    
    pipeline = make_pipeline(col_pipeline, Ridge(alpha=ld, random_state=0))
    pipeline.fit(Xtrain, Ytrain)

    val_pred = np.asarray(pipeline.predict(Xval))
    val_error = np.square(val_pred - Yval)
    val_rmse = np.sqrt(np.sum(val_error) /len(Xval))
    train_rmses.append(val_rmse)
  
  mean_train_rmse = np.mean(train_rmses)
  sd_train_rmse = np.std(train_rmses)

  results_data["model_name"].append(model_name)
  results_data["hyperparamter_setting"].append(hyp)
  results_data["mean_training_rmse"].append(mean_train_rmse)
  results_data["sd_training_rmse"].append(sd_train_rmse)

  print(ld)
  pipeline = make_pipeline(col_pipeline, Ridge(alpha=ld, random_state=0))
  pipeline.fit(total_Xtrain, total_Ytrain)
  test_pred = np.asarray(pipeline.predict(total_Xtest))
  test_error = np.square(test_pred - total_Ytest)
  test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))
  results_data["test_rmse"].append(test_rmse)

1e-05
0.0001
0.001
0.01
0.1
1
10
100
1000
10000
100000


In [23]:
print(results_data)
dfs = pd.DataFrame(results_data)
dfs

{'model_name': ['DTR', 'DTR', 'DTR', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge'], 'hyperparamter_setting': ['5', '10', '20', '1e-05', '0.0001', '0.001', '0.01', '0.1', '1', '10', '100', '1000', '10000', '100000'], 'mean_training_rmse': [1.208094186519705, 1.2454427211400911, 1.3349907701785437, 1.1715478320753452, 1.1715512711066007, 1.171536437922751, 1.1714605324969243, 1.171040718706792, 1.167321905354776, 1.160131189622231, 1.1786003174802102, 1.209263437412607, 1.2260398289555752, 1.2377211414428124], 'sd_training_rmse': [0.02330679904192824, 0.02597186157714203, 0.015919245228671112, 0.023958436587093457, 0.02397211101969935, 0.02395653115905334, 0.023987995722296276, 0.023995313357225725, 0.024521937847497784, 0.026596518399162168, 0.027792669667579175, 0.027734762430704895, 0.02837208791583268, 0.028979845092785294], 'test_rmse': [1.2021076225285747, 1.2622793067391083, 1.372671998450443, 1.1757422813209202, 1.1757509695919

,model_name,hyperparamter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,DTR,5,1.208094,0.023307,1.202108
1,DTR,10,1.245443,0.025972,1.262279
2,DTR,20,1.334991,0.015919,1.372672
3,ridge,1e-05,1.171548,0.023958,1.175742
4,ridge,0.0001,1.171551,0.023972,1.175751
5,ridge,0.001,1.171536,0.023957,1.175457
6,ridge,0.01,1.171461,0.023988,1.175521
7,ridge,0.1,1.171041,0.023995,1.175179
8,ridge,1,1.167322,0.024522,1.172270
9,ridge,10,1.160131,0.026597,1.161836


In [24]:
## K-NEAREST NEIGHBOR

from sklearn.neighbors import KNeighborsRegressor

ks = [1, 5, 10, 15]
for k in ks:
  model_name = "KNN"
  hyp = str(k)
  train_rmses = []

  for nfold in range(5):
  
    train_set = pd.DataFrame()
    val_set = pd.DataFrame()

    for i in range(5):
      if (i != nfold):
        train_set = pd.concat([train_set, data_folds[i]], axis=0)
      else:
        val_set = pd.concat([val_set, data_folds[i]], axis=0)

    Xtrain = train_set[all_features]
    Ytrain = train_set["score"]
    Xval = val_set[all_features]
    Yval = val_set["score"]

    
    pipeline = make_pipeline(col_pipeline, KNeighborsRegressor(n_neighbors=k))
    pipeline.fit(Xtrain, Ytrain)

    val_pred = np.asarray(pipeline.predict(Xval))
    val_error = np.square(val_pred - Yval)
    val_rmse = np.sqrt(np.sum(val_error) /len(Xval))
    train_rmses.append(val_rmse)
  
  mean_train_rmse = np.mean(train_rmses)
  sd_train_rmse = np.std(train_rmses)

  results_data["model_name"].append(model_name)
  results_data["hyperparamter_setting"].append(hyp)
  results_data["mean_training_rmse"].append(mean_train_rmse)
  results_data["sd_training_rmse"].append(sd_train_rmse)

  print(k)
  pipeline = make_pipeline(col_pipeline, KNeighborsRegressor(n_neighbors=k))
  pipeline.fit(total_Xtrain, total_Ytrain)
  test_pred = np.asarray(pipeline.predict(total_Xtest))
  test_error = np.square(test_pred - total_Ytest)
  test_rmse = np.sqrt(np.sum(test_error) /len(Xtest))
  results_data["test_rmse"].append(test_rmse)

1
5
10
15


In [25]:
print(results_data)
dfs = pd.DataFrame(results_data)
dfs

{'model_name': ['DTR', 'DTR', 'DTR', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'ridge', 'KNN', 'KNN', 'KNN', 'KNN'], 'hyperparamter_setting': ['5', '10', '20', '1e-05', '0.0001', '0.001', '0.01', '0.1', '1', '10', '100', '1000', '10000', '100000', '1', '5', '10', '15'], 'mean_training_rmse': [1.208094186519705, 1.2454427211400911, 1.3349907701785437, 1.1715478320753452, 1.1715512711066007, 1.171536437922751, 1.1714605324969243, 1.171040718706792, 1.167321905354776, 1.160131189622231, 1.1786003174802102, 1.209263437412607, 1.2260398289555752, 1.2377211414428124, 1.5984157064644073, 1.2617477639934884, 1.2183164145809222, 1.206307123765442], 'sd_training_rmse': [0.02330679904192824, 0.02597186157714203, 0.015919245228671112, 0.023958436587093457, 0.02397211101969935, 0.02395653115905334, 0.023987995722296276, 0.023995313357225725, 0.024521937847497784, 0.026596518399162168, 0.027792669667579175, 0.027734762430704895, 0.02837208791583268, 0.

,model_name,hyperparamter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,DTR,5,1.208094,0.023307,1.202108
1,DTR,10,1.245443,0.025972,1.262279
2,DTR,20,1.334991,0.015919,1.372672
3,ridge,1e-05,1.171548,0.023958,1.175742
4,ridge,0.0001,1.171551,0.023972,1.175751
5,ridge,0.001,1.171536,0.023957,1.175457
6,ridge,0.01,1.171461,0.023988,1.175521
7,ridge,0.1,1.171041,0.023995,1.175179
8,ridge,1,1.167322,0.024522,1.172270
9,ridge,10,1.160131,0.026597,1.161836


1.4.1 Report, for each model, the hyper parameter setting that resulted in the best performance

ANSWER:   **DTR: 5, Ridge: 10, KNN: 10**

1.4.2 Which model performed the best overall on the cross-validation?

ANSWER:   **Ridge Regression**


1.4.3 Which model performed the best overall on the final test set?

ANSWER:   **Ridge Regression**

1.4.4 With respect to your answer for 1.4.3, why do you think that might be? 
(Note: there is more than one correct way to answer this question)

ANSWER:   **Ridge regression provides regularization.**

1.4.5 Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?

ANSWER:   **Ridge 10^5**

1.4.6 With respect to your answer for 1.4.6, why do you think that might be? (Note: there is more than one correct way to answer this question)

ANSWER:   **Huge Penalty**

# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [26]:
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    if len(w.shape) == 1:
        w = w[:,np.newaxis]
    error = np.mean(np.log(1 + np.exp(-y*np.dot(X,w))))
    return error

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    if len(w.shape) == 1:
        w = w[:,np.newaxis]
    # n = len(X)
    # num = (np.dot(y.T, X))
    deno = (1 + np.exp(y*np.dot(X, w)))
    
    gradient = -1*np.mean((y/deno)*X, axis=0)
    return np.squeeze(gradient)


def run_gradient_descent(X,y):
    old_w = np.array([-1]*(X.shape[1]))
    # change this value! This is an unreasonable step size
    step_size = .01
    new_w = old_w - 1
    
    while ((new_w-old_w)**2).sum() > .0000000001:
        #IMPLEMENT THIS!
        gradient = logistic_gradient(new_w, X, y[:,np.newaxis])
        old_w = new_w
        new_w = old_w - step_size*gradient
    return new_w


In [27]:
import numpy as np
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [28]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.44921366,  3.24682596,  0.19024649]])

In [29]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()

0.8571412494553726

In [30]:
y.shape
#y.reshape(-1,1).shape

(10000,)

## <span style="color:red"> 574 Only</span> Part 2.2 - Optimization with Newton-Raphson <span style="color:red"> 574 Only</span>

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [31]:
def logistic_hessian(w, X, y):

    #print("CALLED HESSIAN")
    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    if len(w.shape) == 1:
        w = w[:,np.newaxis]
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    n = float(X.shape[0])
    en = np.exp(y*np.dot(X,w))
    hessian = (1/n)*(np.dot(X.T, X*(en/np.power(1+en,2))))

    #print(hessian)
    return hessian

In [32]:
from scipy.optimize import minimize
import math

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    
    # note: this is *almost* what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(logistic_objective, 
                    w_init, 
                    jac=logistic_gradient, 
                    hess=logistic_hessian, 
                    args=args,
                    method='Newton-CG',
                    options=opts)

    w = np.transpose(np.array(soln.x))
    w = w[:,np.newaxis]
    return w
    
run_newton_raphson(X,y)


array([[-8.82760446],
       [ 3.40159592],
       [ 0.19809853]])